In [1]:
import os
import sys
import time
import matplotlib as mpl 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd

import tensorflow as tf
from tensorflow import keras

# print(tf.__version__)
# print(sys.version_info)
# for module in mpl,np,pd,sklearn,tf,keras:
#     print(module.__name__,module.__version__)

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

import pprint
pprint.pprint(housing.data[0:5])
pprint.pprint(housing.target[0:5]) 

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:

from sklearn.model_selection import train_test_split
x_train_all,x_test,y_train_all,y_test = train_test_split(housing.data,housing.target,random_state =7)
x_train,x_valid,y_train,y_valid = train_test_split(x_train_all,y_train_all,random_state = 11)
# print(x_valid.shape,y_valid.shape)
# print(x_train.shape,y_train.shape)
# print(x_test.shape,y_test.shape)
# print(np.max(x_train),np.min(x_train))

In [4]:
# x = (x-u均值)/ std方差
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)
print(np.max(x_train_scaled),np.min(x_train_scaled))

81.80221745207753 -2.3594398206242224


In [5]:
# 子类api
class WideDeepModel(keras.models.Model):
    def __init__(self):
        super(WideDeepModel,self).__init__()
        """定义模型的层次"""
        self.hidden1_layer = keras.layers.Dense(30,activation="relu")
        self.hidden2_layer = keras.layers.Dense(30,activation="relu")
        self.output_layer = keras.layers.Dense(1)
        
    def call(self,input):
        """完成模型的正向计算"""
        hidden1 = self.hidden1_layer(input)
        hidden2 = self.hidden2_layer(hidden1)
        concat = keras.layers.concatenate([input,hidden2])
        output = self.output_layer(concat)
        return output
model = WideDeepModel()
# model = keras.models.Sequential([
#     WideDeepModel()
# ])
model.build(input_shape=(None,8))
model.summary()

model.compile(loss="mean_squared_error",
              optimizer="sgd")


Model: "wide_deep_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  270       
_________________________________________________________________
dense_1 (Dense)              multiple                  930       
_________________________________________________________________
dense_2 (Dense)              multiple                  39        
Total params: 1,239
Trainable params: 1,239
Non-trainable params: 0
_________________________________________________________________


In [6]:

callbacks = [
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-2)
]

history=model.fit(x_train_scaled,y_train,
                  epochs = 100,
                  validation_data = (x_valid_scaled,y_valid),
                  callbacks = callbacks)

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 128us/sample - loss: 0.9545 - val_loss: 1.9598
Epoch 2/100
11610/11610 [==============================] - 1s 70us/sample - loss: nan - val_loss: nan
Epoch 3/100
11610/11610 [==============================] - 1s 69us/sample - loss: nan - val_loss: nan
Epoch 4/100
11610/11610 [==============================] - 1s 72us/sample - loss: nan - val_loss: nan
Epoch 5/100
11610/11610 [==============================] - 1s 70us/sample - loss: nan - val_loss: nan
Epoch 6/100
11610/11610 [==============================] - 1s 74us/sample - loss: nan - val_loss: nan


In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,3)
    plt.show()
    
plot_learning_curves(history)

In [ ]:
model.evaluate(x_test_scaled,y_test)